El objetivo de las siguientes funciones es entrenar una red neuronal capaz de predecir posiciones mediante diferencias temporales de tiempo arribo a micrófonos. Para el entrenamiento de la red se generan numéricamente grillas de puntos, se calculan los delays temporales esperados (más un error aleatorio) y se procede a al entrenamiento.

In [ ]:
import numpy as np
from keras import models
from keras import layers
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import tensorflow as tf
import math
import random
from tensorflow.keras.models import load_model
tf.compat.v1.enable_eager_execution()
import seaborn as sns
sns.set_theme()

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Definimos la funciones que vamos a usar para generar datos de entrenamientos y testeo.


def delays_entrenamiento(radio, paso):
  total_samples = int(((2*radio)/paso)**2)
  
  
  M1 = np.array([-3.5,  3.5, 0]) # así es mejor para introducir las posiciones de 
  M2 = np.array([-3.5, -3.5, 0]) # los MIC´s si no están en un cuadrado
  M3 = np.array([ 3.5,  3.5, 0])
  M4 = np.array([ 3.5, -3.5, 0])


  #Otros Parámetros
  v= 350.0 # velocidad del sonido
  fs = 44100.0 # frecuencia de muestreo de las grabaciones
  error = 2/fs
  Z_min =0.5
  Data = np.zeros([total_samples,6]) # delays para 4 MIC´s
  XYZ = np.zeros([total_samples,3]) # posiciones
  s=0
  for i in range(0, int(np.sqrt(total_samples))): # esta es otra forma, Gabo construye para 
      for j in range(0, int(np.sqrt(total_samples))): # recorrer equiespaciado una grilla
          for k in range(1,2): # alto del árbol
              x=-radio + paso*i 
              y=-radio + paso*j  #pasos en la grilla de 2
              z=Z_min*k
                        
              r1=math.sqrt((x-M1[0])**2+(y-M1[1])**2+(z-M1[2])**2)
              r2=math.sqrt((x-M2[0])**2+(y-M2[1])**2+(z-M2[2])**2)
              r3=math.sqrt((x-M3[0])**2+(y-M3[1])**2+(z-M3[2])**2)
              r4=math.sqrt((x-M4[0])**2+(y-M4[1])**2+(z-M4[2])**2) # distancias a cada MIC
                              
              delta12=(1/v)*(r1-r2)+(error)*random.uniform(0,1)
              delta13=(1/v)*(r1-r3)+(error)*random.uniform(0,1)
              delta14=(1/v)*(r1-r4)+(error)*random.uniform(0,1)
              delta23=(1/v)*(r2-r3)+(error)*random.uniform(0,1)
              delta24=(1/v)*(r2-r4)+(error)*random.uniform(0,1)
              delta34=(1/v)*(r3-r4)+(error)*random.uniform(0,1) # diferencias de tiempo, + un error en la cuantificacion
              Data[s,0]=delta12; Data[s,1]=delta13; Data[s,2]=delta14;
              Data[s,3]=delta23; Data[s,4]=delta24
              Data[s,5]=delta34
              XYZ[s,0]=x; XYZ[s,1]=y; XYZ[s,2]=z
              s=s+1
  return Data, XYZ, total_samples
              
           


def delays_testeo(x,y,z):
  M1 = np.array([-3.5,  3.5, 0]) # así es mejor para introducir las posiciones de
  M2 = np.array([-3.5, -3.5, 0]) # los MIC´s si no están en un cuadrado
  M3 = np.array([ 3.5,  3.5, 0])
  M4 = np.array([ 3.5, -3.5, 0])
  #Otros Parámetros
  v= 350.0 # velocidad del sonido
  fs = 44100.0 # frecuencia de muestreo de las grabaciones
  error = 2/fs
  Data = np.zeros([len(x),6]) # delays para 4 MIC´s
  XYZ = np.zeros([len(x),3]) # posiciones

  for i in range(len(x)):
    r1=math.sqrt((x[i]-M1[0])**2+(y[i]-M1[1])**2+(z[i]-M1[2])**2)
    r2=math.sqrt((x[i]-M2[0])**2+(y[i]-M2[1])**2+(z[i]-M2[2])**2)
    r3=math.sqrt((x[i]-M3[0])**2+(y[i]-M3[1])**2+(z[i]-M3[2])**2)
    r4=math.sqrt((x[i]-M4[0])**2+(y[i]-M4[1])**2+(z[i]-M4[2])**2) # distancias a cada MIC
            
    delta12=(1/v)*(r1-r2)+(error)*random.uniform(0,1)
    delta13=(1/v)*(r1-r3)+(error)*random.uniform(0,1)
    delta14=(1/v)*(r1-r4)+(error)*random.uniform(0,1)
    delta23=(1/v)*(r2-r3)+(error)*random.uniform(0,1)
    delta24=(1/v)*(r2-r4)+(error)*random.uniform(0,1)
    delta34=(1/v)*(r3-r4)+(error)*random.uniform(0,1) # diferencias de tiempo, + un error en la cuantificacion
    Data[i,0]=delta12; Data[i,1]=delta13; Data[i,2]=delta14;
    Data[i,3]=delta23; Data[i,4]=delta24
    Data[i,5]=delta34
    XYZ[i,0]=x[i]; XYZ[i,1]=y[i]; XYZ[i,2]=z[i]
  return Data, XYZ



In [ ]:
Data, XYZ, total_samples= delays_entrenamiento(6,.35) #creo los delays de entrenamiento
np.savetxt('/content/drive/MyDrive/RedGabo_Felipe & Constanza/Data.txt',Data,fmt = '%10.5f') #los guardo
np.savetxt('/content/drive/MyDrive/RedGabo_Felipe & Constanza/XYZ.txt',XYZ,fmt = '%10.5f') #los cargo

train_data=np.loadtxt('/content/drive/MyDrive/RedGabo_Felipe & Constanza/Data.txt') #asigno los train targets y train data
train_targets=np.loadtxt('/content/drive/MyDrive/RedGabo_Felipe & Constanza/XYZ.txt')



delays_medidos=np.zeros(train_data.shape[1])
train_data.shape


In [ ]:
#entrenamos la red
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(3))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

# dio muy bien con 64 ...
#modelx = build_model()
#modelx.fit(train_data, train_targetsx,epochs=80, batch_size=16, verbose=0)
#modely = build_model()
#modely.fit(train_data, train_targetsy,epochs=80, batch_size=16, verbose=0)
#modelz = build_model()
#modelz.fit(train_data, train_targetsz,epochs=80, batch_size=16, verbose=0)

k = 3 #4
num_val_samples = len(train_data) // k
num_epochs = 1000 #10
all_mae_histories = []

# para los x, los validation

for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
         axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
         axis=0)
    model = build_model()
    history=model.fit(partial_train_data, partial_train_targets,
                      validation_data=(val_data, val_targets),
                      epochs=num_epochs, batch_size=1, verbose=0)
    
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    mae_history = history.history['mae']
    all_mae_histories.append(mae_history)

model.save('/content/drive/MyDrive/RedGabo_Felipe & Constanza/(6412812864)_r.h5') #guarden el modelo en su Drive personal
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]


plt.plot(range(1, len(average_mae_history) + 1), average_mae_history,'.')
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.ylim(np.min(average_mae_history)-1,np.min(average_mae_history)+1)
plt.hlines(np.min(average_mae_history),0,num_epochs,colors='r', linestyles='dashed')
plt.show()
print('El MAE minimo es: ',np.min(average_mae_history))

In [ ]:
#probamos la red entrenada en posiciones determinadas
x = [1,.5,2,4,-2]#Introducir las posiciones en formato de arreglo
y = [2,1,-1,.79,-2]
z = [0,0,0,0,0]
delays_UnaPos, XYZ_UnaPos= delays_testeo(x,y,z)
np.savetxt('/content/drive/My Drive/RedGabo_Felipe & Constanza/delays_UnaPos.txt', delays_UnaPos,fmt = '%10.5f') #/content/drive/My Drive/XYZ.txt
np.savetxt('/content/drive/MyDrive/RedGabo_Felipe & Constanza/XYZ_UnaPos.txt', XYZ_UnaPos,fmt = '%10.5f') #/content/drive/My Drive/XYZ.txt

np.set_printoptions(precision=3, suppress=True) #para quitar la notación científica

# definir el vector de los delays
selected_data=np.loadtxt('/content/drive/MyDrive/RedGabo_Felipe & Constanza/delays_UnaPos.txt')
xyz=model.predict(selected_data) #pasamos los datos por la red
print('Posiciones predecidas')
print('delays',selected_data.shape)
print(xyz)

pos_reales=np.loadtxt('/content/drive/MyDrive/RedGabo_Felipe & Constanza/XYZ_UnaPos.txt')
print()
print('Posiciones reales')
print('pos',pos_reales.shape)
print(pos_reales)